In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
import glob
import os
import pickle
import json

import cv2
import numpy as np

In [3]:
from skimage.feature import hog

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

In [4]:
DATA_DIR = os.path.join('../data')
VEHICLES = os.path.join(DATA_DIR, 'vehicles')
NON_VEHICLES = os.path.join(DATA_DIR, 'non-vehicles')
DIRS = ["GTI_Far", "GTI_Left", "GTI_MiddleClose", "GTI_Right"]

In [5]:
def hog_features(img):
    rhf = hog(img[:,:,0], orientations=9, pixels_per_cell=(8, 8), 
                     cells_per_block=(2, 2), feature_vector=True)
    ghf = hog(img[:,:,1], orientations=9, pixels_per_cell=(8, 8), 
                     cells_per_block=(2, 2), feature_vector=True)
    bhf = hog(img[:,:,2], orientations=9, pixels_per_cell=(8, 8), 
                     cells_per_block=(2, 2), feature_vector=True)
    return np.hstack((rhf, ghf, bhf))

In [6]:
def read(data_dir, extract_feature):
    for img_path in glob.iglob(os.path.join(data_dir, 'GTI*', '*.png')):
        img = cv2.imread(img_path).astype(np.float32)/255
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        yield extract_feature(img)

In [7]:
vehicles = [feature for feature in read(VEHICLES, hog_features)]

In [8]:
non_vehicles = [feature for feature in read(NON_VEHICLES, hog_features)]

In [9]:
data = np.vstack((vehicles, non_vehicles))
labels = np.hstack((np.ones(len(vehicles), np.int32), np.zeros(len(non_vehicles), np.int32)))

In [10]:
len(labels) == len(data)

True

In [11]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [12]:
x_train.shape

(5380, 5292)

In [13]:
clf = svm.LinearSVC()
scaler = StandardScaler()
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [14]:
acc = pipeline.score(x_test, y_test)

In [15]:
acc

0.98068350668647841

In [16]:
pred = pipeline.predict(x_test)

In [17]:
cm = confusion_matrix(y_test, pred)

In [18]:
cm

array([[785,  11],
       [ 15, 535]])

In [19]:
with open('classifier.p', 'wb') as _file:
    pickle.dump(pipeline, _file)

In [20]:
with open('classifier.p', 'rb') as _file:
    pipex = pickle.load(_file)

In [21]:
pred = pipex.predict(x_test)
cm = confusion_matrix(y_test, pred)
cm

array([[785,  11],
       [ 15, 535]])